Après avoir assimilé le cours sur les réseaux de neurones, nous avons pu commencé la première semaine en se fixant comme objectif la création d'un programme capable de générer des automates aléatoires en utilisant un langage totalement aléatoire. Pour cela, nous avons utilisé la classe Automate de la bibliothèque "automata-lib", qui nous a permis de bénéficier de fonctions facilitant grandement notre travail par la suite. Nous avons ensuite créé une nouvelle classe Automate, héritant de la précédente, en y ajoutant des méthodes pour la rendre plus adaptée à notre projet.

In [2]:
import random
from automata.fa.nfa import NFA
from automata.fa.gnfa import GNFA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import exrex
import numpy as np
import pickle

Une fois que les imports nécéssaire pour faire tourner la suite du code a était faite, nous avons pu commencer a écrire l'algorithme qui nous permetterai de générer un automate aléatoire.

l'appel de la fonction random_automaton nécéssite un certain nombre se parametre. On peut choisir l'alphabet que l'on souhaite utiliser pour l'automate, le nombre minimum et maximum d'états que l'on veut, ainsi que la densité de transitions. On peut également choisir le nombre d'états initiaux et finaux que l'on souhaite, ainsi qu'une plage de variation pour chaque afin d'assurer un réel aléatoire.

Ensuite, la fonction utilise ces paramètres pour créer un automate avec des transitions et des états initiaux et finaux générés aléatoirement. La densité de transition étant un nombre compris entre 0 et 1, elle représente la probabilité prise pour relier tous les états avec chaque lettre de notre alphabet d'états. Le résultat renvoyé est une structure de données contenant les états, l'alphabet, les transitions, et les états initiaux et finaux de l'automate.

le paramètre seed est donné à notre fonction pour obtenir un automate aléatoire qui a déjà été généré dans le cas où l'on voudrait retrouver un automate spécifique.

Cette fonction constituera la base de notre training-set pour le réseaux de neurones et nous sera utile pour étudier le comportement des automates aléatoires.

In [3]:
class Automaton(NFA):
    def __init__(
        self, states, input_symbols, transitions, initial_state, final_states
    ):
        super().__init__(
            states=states,
            input_symbols=input_symbols,
            transitions=transitions,
            initial_state=initial_state,
            final_states=final_states,
        )

    def __setattr__(self, name, value):
        """Overrides __setattr__ to allow making other attributes"""
        pass

    @staticmethod
    def random_automaton(
        alphabet,
        min_states,
        max_states,
        trans_density,
        init_amount,
        final_amount,
        init_range=0,
        final_range=0,
        seed=None,
    ):
        """
        This function creates a random
        automaton with desired probabilities
        :param alphabet: set of the alphabet used by the automaton (set of str)
        :param min_states: minimum amount of states (int)
        :param max_states: maximum amount of states (int)
        :param trans_density: chance for a transition
        to be created (percentage)
        :param init_amount: amount of initial states (int)
        :param final_amount: amount of final states (int)
        :param init_range: maximal variation
        in the amount of initial states (int)
        :param final_range: maximal variation in
        the amount of final states (int)
        :param seed: random seed, None by default
        :return: a set of states, a dictionary of transition,
        set of initial states, set of final states
        """
        if seed is not None:
            random.seed(seed)
        states = {"init"}
        for i in range(random.randint(min_states, max_states)):
            states.add(i)
        trans = {"init": {"": set()}}
        for state_in in states.difference({"init"}):
            for state_out in states.difference({"init"}):
                for char in alphabet:
                    if random.random() < trans_density:
                        try:
                            trans[state_in][char].add(state_out)
                        except KeyError:
                            try:
                                trans[state_in][char] = {state_out}
                            except KeyError:
                                trans[state_in] = {char: {state_out}}
        init = set()
        for i in range(
            max(
                init_amount + random.randint(-init_range, init_range),
                1,
            )
        ):
            if len(states.difference(init.union({"init"}))) > 0:
                init.add(
                    random.choice(
                        list(states.difference(init.union({"init"})))
                    )
                )
            else:
                break
        for i in init:
            trans["init"][""].add(i)
        final = set()
        for i in range(
            max(
                final_amount + random.randint(-final_range, final_range),
                0,
            )
        ):
            if len(states.difference(final.union({"init"}))) > 0:
                final.add(
                    random.choice(
                        list(states.difference(final.union({"init"})))
                    )
                )
            else:
                break
        return Automaton(states, alphabet, trans, "init", final)

On veut maintenant pouvoir visualiser et sauvegarder les automates qui sont générés. Les fonctions save et load permettent de sauvegarder un automate dans un fichier pickle et de le recréer à partir de ce même fichier. Un fichier pickle est un fichier qui permet de stocker des paramètres, des variables ou, dans notre cas, des objets pour qu'ils soient facilement interprétables et utilisables en Python. La méthode str a été modifiée pour les objets de la classe afin de permettre l'affichage et l'enregistrement de l'automate sous forme de graphe.

In [4]:
class Automaton(NFA):
    def __init__(
        self, states, input_symbols, transitions, initial_state, final_states
    ):
        super().__init__(
            states=states,
            input_symbols=input_symbols,
            transitions=transitions,
            initial_state=initial_state,
            final_states=final_states,
        )

    def __setattr__(self, name, value):
        """Overrides __setattr__ to allow making other attributes"""
        pass

    @staticmethod
    def random_automaton(
        alphabet,
        min_states,
        max_states,
        trans_density,
        init_amount,
        final_amount,
        init_range=0,
        final_range=0,
        seed=None,
    ):
        """
        This function creates a random
        automaton with desired probabilities
        :param alphabet: set of the alphabet used by the automaton (set of str)
        :param min_states: minimum amount of states (int)
        :param max_states: maximum amount of states (int)
        :param trans_density: chance for a transition
        to be created (percentage)
        :param init_amount: amount of initial states (int)
        :param final_amount: amount of final states (int)
        :param init_range: maximal variation
        in the amount of initial states (int)
        :param final_range: maximal variation in
        the amount of final states (int)
        :param seed: random seed, None by default
        :return: a set of states, a dictionary of transition,
        set of initial states, set of final states
        """
        if seed is not None:
            random.seed(seed)
        states = {"init"}
        for i in range(random.randint(min_states, max_states)):
            states.add(i)
        trans = {"init": {"": set()}}
        for state_in in states.difference({"init"}):
            for state_out in states.difference({"init"}):
                for char in alphabet:
                    if random.random() < trans_density:
                        try:
                            trans[state_in][char].add(state_out)
                        except KeyError:
                            try:
                                trans[state_in][char] = {state_out}
                            except KeyError:
                                trans[state_in] = {char: {state_out}}
        init = set()
        for i in range(
            max(
                init_amount + random.randint(-init_range, init_range),
                1,
            )
        ):
            if len(states.difference(init.union({"init"}))) > 0:
                init.add(
                    random.choice(
                        list(states.difference(init.union({"init"})))
                    )
                )
            else:
                break
        for i in init:
            trans["init"][""].add(i)
        final = set()
        for i in range(
            max(
                final_amount + random.randint(-final_range, final_range),
                0,
            )
        ):
            if len(states.difference(final.union({"init"}))) > 0:
                final.add(
                    random.choice(
                        list(states.difference(final.union({"init"})))
                    )
                )
            else:
                break
        return Automaton(states, alphabet, trans, "init", final)

    def __str__(self):
        try:
            open("./display.png", "x")
        except FileExistsError:
            pass
        self.show_diagram(path="./display.png")
        plt.figure("Automate")
        plt.axis("off")
        plt.imshow(mpimg.imread("display.png"))
        plt.show()
        return super.__str__(self)



    def save_automaton(self, name):
        """
        This function saves the automaton in a pickle file
        :param name: name of the automaton (str)
        :return: None
        """
        file_name = "{}.pkl".format(name)
        with open(file_name, "wb") as file:
            pickle.dump(
                (
                    self.states,
                    self.input_symbols,
                    self.transitions,
                    self.initial_state,
                    self.final_states,
                ),
                file,
            )
            print("Automaton saved in {}".format(file_name))

    @staticmethod
    def load_automaton(file_name):
        """
        This function load a saved automaton
        :param file_name: name of the file containing the automaton (str)
        :return: Automaton object
        """
        with open(file_name, "rb") as file:
            info = pickle.load(file)
            return Automaton(info[0], info[1], info[2], info[3], info[4])

In [7]:

alphabet = {"a", "b"}
min_states = 5
max_states = 10
trans_density = 0.12
init_amount = 2
final_amount = 2
init_range = 1
final_range = 1
automaton = Automaton.random_automaton(
    alphabet,
    min_states,
    max_states,
    trans_density,
    init_amount,
    final_amount,
    init_range,
    final_range,
)

print(automaton)

FileNotFoundError: [WinError 2] "dot" not found in path.

Par la suite, nous aimerions pouvoir générer un nombre défini de mots grâce à l'expression régulière de l'automate. Ce sont ces mêmes mots qui serviront d'entraînement à notre réseau de neurones. En effet, nous souhaitons entraîner notre IA avec un ensemble de mots autorisés par l'automate ainsi qu'avec des mots qui ne le sont pas. Cela permettra à notre IA de décider si l'automate acceptera ou non un mot saisi en entrée, sans même connaître la forme ou l'expression régulière de l'automate.

Pour cela, nous devrons générer ces mots, puis les mettre sous le format "one hot". L'encodage one hot est une technique couramment utilisée pour représenter des catégories sous forme de vecteurs binaires. Dans notre cas, chaque catégorie représente une lettre de notre alphabet, et nous voulons encoder nos mots en tant que vecteurs one hot. Par exemple, si notre alphabet se compose des lettres a et b, nous pouvons représenter le mot aba comme suit :
- a = [1, 0]
- b = [0, 1]
- aba = [[1, 0], [0, 1], [1, 0]]

De cette façon, chaque lettre est représentée par un vecteur binaire distinct et les mots peuvent être encodés de manière unique.

La fonction "classify_words()" prend en paramètre le nombre de mots souhaité. En lui donnant l'expression régulière de Sigma étoile, elle génère des mots de longueur comprise entre 0 et 100. Ces mots sont ensuite encodés en "one hot", puis nous vérifions s'ils sont acceptés par notre automate afin de les tagger avec un 1 s'ils le sont, et un 0 sinon.


In [ ]:
class Automaton(NFA):
    def __init__(
        self, states, input_symbols, transitions, initial_state, final_states
    ):
        super().__init__(
            states=states,
            input_symbols=input_symbols,
            transitions=transitions,
            initial_state=initial_state,
            final_states=final_states,
        )

    def __setattr__(self, name, value):
        """Overrides __setattr__ to allow making other attributes"""
        pass

    @staticmethod
    def random_automaton(
        alphabet,
        min_states,
        max_states,
        trans_density,
        init_amount,
        final_amount,
        init_range=0,
        final_range=0,
        seed=None,
    ):
        """
        This function creates a random
        automaton with desired probabilities
        :param alphabet: set of the alphabet used by the automaton (set of str)
        :param min_states: minimum amount of states (int)
        :param max_states: maximum amount of states (int)
        :param trans_density: chance for a transition
        to be created (percentage)
        :param init_amount: amount of initial states (int)
        :param final_amount: amount of final states (int)
        :param init_range: maximal variation
        in the amount of initial states (int)
        :param final_range: maximal variation in
        the amount of final states (int)
        :param seed: random seed, None by default
        :return: a set of states, a dictionary of transition,
        set of initial states, set of final states
        """
        if seed is not None:
            random.seed(seed)
        states = {"init"}
        for i in range(random.randint(min_states, max_states)):
            states.add(i)
        trans = {"init": {"": set()}}
        for state_in in states.difference({"init"}):
            for state_out in states.difference({"init"}):
                for char in alphabet:
                    if random.random() < trans_density:
                        try:
                            trans[state_in][char].add(state_out)
                        except KeyError:
                            try:
                                trans[state_in][char] = {state_out}
                            except KeyError:
                                trans[state_in] = {char: {state_out}}
        init = set()
        for i in range(
            max(
                init_amount + random.randint(-init_range, init_range),
                1,
            )
        ):
            if len(states.difference(init.union({"init"}))) > 0:
                init.add(
                    random.choice(
                        list(states.difference(init.union({"init"})))
                    )
                )
            else:
                break
        for i in init:
            trans["init"][""].add(i)
        final = set()
        for i in range(
            max(
                final_amount + random.randint(-final_range, final_range),
                0,
            )
        ):
            if len(states.difference(final.union({"init"}))) > 0:
                final.add(
                    random.choice(
                        list(states.difference(final.union({"init"})))
                    )
                )
            else:
                break
        return Automaton(states, alphabet, trans, "init", final)

    def __str__(self):
        try:
            open("./display.png", "x")
        except FileExistsError:
            pass
        self.show_diagram(path="./display.png")
        plt.figure("Automate")
        plt.axis("off")
        plt.imshow(mpimg.imread("display.png"))
        plt.show()
        return super.__str__(self)

    def save_automaton(self, name):
        """
        This function saves the automaton in a pickle file
        :param name: name of the automaton (str)
        :return: None
        """
        file_name = "{}.pkl".format(name)
        with open(file_name, "wb") as file:
            pickle.dump(
                (
                    self.states,
                    self.input_symbols,
                    self.transitions,
                    self.initial_state,
                    self.final_states,
                ),
                file,
            )
            print("Automaton saved in {}".format(file_name))

    @staticmethod
    def load_automaton(file_name):
        """
        This function load a saved automaton
        :param file_name: name of the file containing the automaton (str)
        :return: Automaton object
        """
        with open(file_name, "rb") as file:
            info = pickle.load(file)
            return Automaton(info[0], info[1], info[2], info[3], info[4])
    
    def get_one_hot_index(self):
        return {
            list(self.input_symbols)[i]: i
            for i in range(len(self.input_symbols))
        }

    def one_hot_encoder(self, word, length):
        """
        This function encode a word in the one-hot format
        :param word: word to encode (str)
        :param length: length of the one-hot array used for encoding
        :return: array with shape (len(self.input_symbols), length) filled with 0 and 1
        """
        encoded = np.zeros((len(self.input_symbols), length))
        for i in range(len(word)):
            encoded[self.get_one_hot_index()[word[i]]][i] = 1
        return encoded

    def one_hot_decoder(self, encoded):
        """
        This function returns the word encoded in a one-hot array
        :param encoded: one-hot array to decode
        :return: decoded word (str)
        """
        word = ""
        one_hot_index = tuple(self.get_one_hot_index().keys())
        for col in range(np.shape(encoded)[1]):
            index = np.argwhere(
                np.hsplit(encoded, np.shape(encoded)[1])[col].reshape(
                    (np.shape(encoded)[0],)
                )
            )
            if np.shape(index)[0]:
                word += one_hot_index[index[0][0]]
            else:
                break
        return word

    def classify_words(self, nb):
        """
        This function creates a 2 array. One with shape (nb, alphabet_length, word_max_length) where nb in the number of
        words classified, alphabet_length is the amount of character in the alphabet and word_max_length is the length
        of the longest word classified. Ths array stores all the words classified encoded in one hot format.
        The second array with shape (nb, ) stores a one in it's x index if the x classified word if accepted by the
        automaton and a 0 otherwise.
        :param nb: number of word to classify
        :return: 2 arrays with shape (nb, alphabet_length, word_max_length) and (nb, ) and dtype=float64
        """
        classified = 0
        sigma_star = Automaton(
            {0},
            self.input_symbols,
            {0: {char: {0} for char in self.input_symbols}},
            0,
            {0},
        ).get_regex()
        one_hot_words = []
        tag = []
        length = round(np.log(nb) / np.log(len(self.input_symbols))) + 1
        for word in exrex.generate(sigma_star, limit=100):
            encoded = self.one_hot_encoder(word, length)
            one_hot_words.append(encoded)
            tag.append(int(self.accepts_input(word)))
            classified += 1
            if classified >= nb:
                return np.array(one_hot_words, dtype="float64"), np.array(
                    tag, dtype="float64"
                )

    def get_regex(self):
        """
        :return: regular expression of the automaton (str)
        """
        return GNFA.from_nfa(self).to_regex()

Une pipeline est une suite d'opérations effectuées successivement sur des données. Elle permet de structurer le traitement des données en plusieurs étapes distinctes, chacune étant réalisée par une fonction ou une méthode spécifique. Dans le cadre de notre projet, nous avons essentiellement terminé la construction de notre pipeline de données pour notre modèle de réseaux de neurones.